In [ ]:
from logging import root
from pathlib import Path

from ai_xp.llm_proxy import AiSummarizer, OpenRouterAiProxy, VideoModel
from ai_xp.scrapper import YouTubeHtmlScrapper
from ai_xp.transcript import TranscriptPath, extract_video_id
from ai_xp.utils import render_timestamp_slug, retrieve_api_key
from ai_xp.database import fetch_one_metadata, fetch_one_transcript
import pandas as pd


video_url = "https://www.youtube.com/watch?v=GgMot4KPQ-o"
video_url = "https://www.youtube.com/watch?v=faoCo4AsmZs"
video_url = "https://www.youtube.com/watch?v=0S-sfdRTxbM"
video_url = "https://www.youtube.com/watch?v=tVaDmeC84KM"
video_url = "https://www.youtube.com/watch?v=WXlPYAV1TfQ"
video_url = "https://www.youtube.com/watch?v=JymeTIeWjLU"
video_url = "https://www.youtube.com/watch?v=x2tj3CE2qQE"
video_url = "https://www.youtube.com/watch?v=YrCDH794Ya8"
video_url = "https://www.youtube.com/watch?v=4SCTCpsKkoU"
video_url = "https://www.youtube.com/watch?v=Sexea1QaTXo"
video_url = "https://www.youtube.com/watch?v=xHdjzeeuP4o"
video_url = "https://www.youtube.com/watch?v=v5YcH5X5Xgo"
video_url = "https://www.youtube.com/watch?v=fZU-pNEKAQk"
video_url = "https://www.youtube.com/watch?v=YcIbZGTRMjI"
video_url = "https://www.youtube.com/watch?v=yXjMpdS1ZWY"

def please_summarize_for_me_because_i_am_lazy(
    video_url: str,
    outputs_lookup_dir_path: Path,
    *,
    preferred_languages: tuple[str, ...] = ("fr", "fr-FR", "en"),
) -> dict[str, Path]:
    video_id = extract_video_id(video_url)
    if video_id is None:
        raise ValueError(f"Failed to extract video_id for {video_url}")

    now = pd.Timestamp.now()

    root_database_path = outputs_lookup_dir_path / "manual"
    root_database_path.mkdir(exist_ok=True, parents=True)

    llm_output_lookup_dir_path = (
        root_database_path / render_timestamp_slug(now) / "llm_output"
    )
    llm_output_lookup_dir_path.mkdir(exist_ok=True, parents=True)

    transcript_dir_path = (
        root_database_path / render_timestamp_slug(now) / "transcript_output"
    )
    transcript_dir_path.mkdir(exist_ok=True, parents=True)

    metadata_dir_path = (
        root_database_path / render_timestamp_slug(now) / "metadata_output"
    )
    metadata_dir_path.mkdir(exist_ok=True, parents=True)

    scrapper = YouTubeHtmlScrapper.from_video_id(video_id)
    if scrapper is None:
        raise ValueError(f"Failed to fetch metadata for {video_id}")

    metadata_path = fetch_one_metadata(metadata_dir_path, video_id, scrapper)
    video = VideoModel.from_path(Path(metadata_path))
    assert video.title

    transcript_path = fetch_one_transcript(
        transcript_dir_path,
        video.video_id,
        video.title,
        preferred_languages=preferred_languages,
    )

    prompts_path = Path("../resources/prompts/prompts.toml")
    secrets_path = Path.home() / Path(".secrets/yt_summary_secrets.json")
    api_key = retrieve_api_key(secrets_path=secrets_path)
    proxy = OpenRouterAiProxy(api_key=api_key)
    summarizer = AiSummarizer.instantiate(
        proxy,
        dry_run=False,
        prompts_path=prompts_path,
        creation_time=None,  # creation time handled manually
    )
    print(summarizer)

    language_code = TranscriptPath.from_path(transcript_path).language_code
    output_paths = summarizer.summarize_with_ai(
        video,
        transcript_path,
        llm_output_lookup_dir_path,
        language_code,
        None,
    )
    for k, v in output_paths.items():
        print(f"{k}: {v}")

    return output_paths

outputs_lookup_dir_path = Path("../generated").resolve()
preferred_languages = ("fr", "fr-FR")  # force french
preferred_languages = ("en",)  # force english
# try french then english, then try translate to french then english
preferred_languages = ("fr", "fr-FR", "en")  
please_summarize_for_me_because_i_am_lazy(
    video_url, outputs_lookup_dir_path, preferred_languages=preferred_languages
)